# Pré-procesamento de Dados do miRWalk

# Importação de Bibliotecas

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

# Constantes

In [2]:
# Caminho da pasta raiz de dados
DATA_PATH = Path.cwd().parent / 'data'

# Caminho da pasta de dados intermediários
INTERIM_DATA_PATH = DATA_PATH / 'interim'

# Caminho da pasta de dados processados
PROCESSED_DATA_PATH = DATA_PATH / 'processed'
PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

# Carregamento de Dados

In [3]:
# Lê o arquivo um arquivo com as interações mineradas no miRWalk
df_interactions = pd.read_csv(
    filepath_or_buffer=Path(INTERIM_DATA_PATH / 'miRWalk-interactions.csv')
)

# Pré-processamento

## Manipulação das Colunas

In [4]:
# Colunas com dados de interesse
columns = [
    'mirnaid',     # id do microRNA
    'refseqid',    # id RNA mensageiro
    'genesymbol',  # id gene
    'bindingp',    # p-valor do algoritmo TarPmiR
    'validated',   # id do miRTarBase
    'TargetScan',  # interação predita pelo TargetScanHuman
    'miRDB'        # interação predita pelo miRDB
]

# Seleciona as colunas com os dados de interesse
df_interactions = df_interactions[columns]

# Remove as interações com repetição
df_interactions = df_interactions.drop_duplicates()

# Cria coluna para indicar se a interação foi validada pelo miRTarBase
df_interactions['miRTarBase'] = np.where(
    df_interactions['validated'].isna(), 0, 1
)

# Transforma os NaNs de `validated` em vazios
df_interactions['validated'] = np.where(
    df_interactions['validated'].isna(), '', df_interactions['validated']
)

## Filtragem das Interações

In [5]:
# Busca pelas interações de interesse
df_interactions = df_interactions \
    .query(
        '(bindingp >= 0.8) and ' +
        '(miRTarBase == 1 or TargetScan == 1 or miRDB == 1)'
    ) \
    .reset_index(drop=True)

# Armazenamento dos Dados

## MicroRNAs

In [6]:
# Salva um arquivo com os microRNAs
df_mirnas = df_interactions[['mirnaid']] \
    .drop_duplicates(ignore_index=True)
df_mirnas.to_csv(path_or_buf=PROCESSED_DATA_PATH / 'mirnas.csv', index=False)

In [7]:
# Imprime o DataFrame de microRNAs
df_mirnas

,mirnaid
0,hsa-miR-495-3p
1,hsa-miR-485-5p
2,hsa-miR-665
3,hsa-miR-539-5p
4,hsa-miR-543
5,hsa-miR-654-5p
6,hsa-miR-370-3p
7,hsa-miR-544a
8,hsa-miR-541-3p
9,hsa-miR-432-5p


## RNAs Mensageiros

In [8]:
# Salva um arquivo com os RNAs mensageiros
df_mrnas = df_interactions[['refseqid', 'genesymbol']] \
    .drop_duplicates(ignore_index=True)
df_mrnas.to_csv(path_or_buf=PROCESSED_DATA_PATH / 'mrnas.csv', index=False)

In [9]:
# Imprime o DataFrame de RNAs mensageiros
df_mrnas

,refseqid,genesymbol
0,NM_001330416,ST8SIA2
1,NM_001330442,MTA3
2,NM_001330443,MTA3
3,NM_001330458,ATL2
4,NM_001330461,ATL2
...,...,...
25628,NM_001376887,TNFSF14
25629,NM_015041,CLUAP1
25630,NM_001256823,ZNF195
25631,NM_019601,SUSD2


## Interações

In [10]:
# Salva um arquivo com as interações de interesse
df_interactions = df_interactions.drop(columns=['genesymbol'])
df_interactions.to_csv(path_or_buf=PROCESSED_DATA_PATH / 'interactions.csv', index=False)

In [11]:
# Imprime o DataFrame de interações de interesse
df_interactions

,mirnaid,refseqid,bindingp,validated,TargetScan,miRDB,miRTarBase
0,hsa-miR-495-3p,NM_001330416,0.846154,,1,1,0
1,hsa-miR-495-3p,NM_001330442,0.846154,MIRT052935,0,1,1
2,hsa-miR-495-3p,NM_001330443,0.846154,MIRT052935,0,1,1
3,hsa-miR-495-3p,NM_001330458,0.846154,,0,1,0
4,hsa-miR-495-3p,NM_001330461,0.846154,,0,1,0
...,...,...,...,...,...,...,...
81873,hsa-miR-496,NM_001300884,1.000000,,0,1,0
81874,hsa-miR-496,NM_001300909,1.000000,,0,1,0
81875,hsa-miR-496,NM_001301212,1.000000,,0,1,0
81876,hsa-miR-496,NM_001324144,1.000000,,0,1,0
